In [1]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Define the date range and categories
dates = pd.date_range(start="2024-01-01", end="2024-07-31", freq='M')
categories = ["Groceries", "Housing", "Health", "Entertainment", "Insurance", "Transportation"]

# Generate dummy spending data
np.random.seed(42)
data = {
    "Date": np.tile(dates, len(categories)),
    "Category": np.repeat(categories, len(dates)),
    "Spending": np.random.uniform(100, 1000, len(dates) * len(categories))
}
df = pd.DataFrame(data)

# Generate dummy CPI data (e.g., simulate some inflation rates)
cpi_data = {
    "Date": dates,
    "CPI": np.random.uniform(1.01, 1.05, len(dates))  # simulate CPI increase between 1% to 5%
}
cpi_df = pd.DataFrame(cpi_data)

# Merge the dataframes
merged_df = pd.merge(df, cpi_df, on="Date")
merged_df["Adjusted Spending"] = merged_df["Spending"] * merged_df["CPI"]

# Display the dataframes
print("Dummy Spending Data:")
display(df)

print("\nCPI Data:")
display(cpi_df)

print("\nMerged Data with Adjusted Spending:")
display(merged_df)

# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'By Category', 'value': 'Category'},
            {'label': 'By Month', 'value': 'Date'}
        ],
        value='Category',
        style={'width': '50%'}
    ),
    dcc.Dropdown(
        id='filter-value',
        style={'width': '50%', 'margin-top': '10px'}
    ),
    dcc.Graph(id='spending-chart')
])

# Callback to update the filter value options based on the filter type
@app.callback(
    Output('filter-value', 'options'),
    [Input('filter-type', 'value')]
)
def set_filter_value_options(filter_type):
    if filter_type == 'Category':
        return [{'label': cat, 'value': cat} for cat in categories]
    else:
        return [{'label': date.strftime("%B %Y"), 'value': date} for date in dates]

# Callback to update the graph based on the selected filter
@app.callback(
    Output('spending-chart', 'figure'),
    [Input('filter-type', 'value'), Input('filter-value', 'value')]
)
def update_graph(filter_type, filter_value):
    if filter_value is None:
        return {}

    if filter_type == 'Category':
        filtered_df = merged_df[merged_df['Category'] == filter_value]
        title = f'Spending and Adjusted Spending for {filter_value}'
        x_column = 'Date'
    else:
        # Convert the string date back to a datetime object if needed
        try:
            filter_value = pd.to_datetime(filter_value)
            filtered_df = merged_df[merged_df['Date'] == filter_value]
            title = f'Spending and Adjusted Spending for {filter_value.strftime("%B %Y")}'
            x_column = 'Category'
        except Exception as e:
            return {}

    fig = px.bar(
        filtered_df,
        x=x_column,
        y=['Spending', 'Adjusted Spending'],
        barmode='group',
        title=title
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


Dummy Spending Data:


,Date,Category,Spending
0,2024-01-31,Groceries,437.086107
1,2024-02-29,Groceries,955.642876
2,2024-03-31,Groceries,758.794548
3,2024-04-30,Groceries,638.792636
4,2024-05-31,Groceries,240.416776
5,2024-06-30,Groceries,240.395068
6,2024-07-31,Groceries,152.275251
7,2024-01-31,Housing,879.558531
8,2024-02-29,Housing,641.003511
9,2024-03-31,Housing,737.265320



CPI Data:


,Date,CPI
0,2024-01-31,1.011376
1,2024-02-29,1.046373
2,2024-03-31,1.020351
3,2024-04-30,1.036501
4,2024-05-31,1.022468
5,2024-06-30,1.030803
6,2024-07-31,1.031868



Merged Data with Adjusted Spending:


,Date,Category,Spending,CPI,Adjusted Spending
0,2024-01-31,Groceries,437.086107,1.011376,442.058198
1,2024-01-31,Housing,879.558531,1.011376,889.563985
2,2024-01-31,Health,263.642470,1.011376,266.641546
3,2024-01-31,Entertainment,225.544475,1.011376,228.110165
4,2024-01-31,Insurance,633.173112,1.011376,640.375799
5,2024-01-31,Transportation,827.557613,1.011376,836.971529
6,2024-02-29,Groceries,955.642876,1.046373,999.958727
7,2024-02-29,Housing,641.003511,1.046373,670.728648
8,2024-02-29,Health,265.064059,1.046373,277.355826
9,2024-02-29,Entertainment,362.930184,1.046373,379.760278
